In [1]:
pip --version


pip 21.3 from /Users/xdharmai/.local/lib/python3.8/site-packages/pip (python 3.8)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install prometheus-api-client

You should consider upgrading via the '/Users/xdharmai/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install matplotlib

You should consider upgrading via the '/Users/xdharmai/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install numpy==1.23.0

You should consider upgrading via the '/Users/xdharmai/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
from prometheus_api_client import *
import matplotlib.pyplot as plt
import os
import numpy as np
import sys
import datetime
import pandas as pd
import seaborn as sns
# Set default Seaborn style
sns.set(style="darkgrid")
sns.set_context("talk", font_scale=1.4)
## hides ipython warnings
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (30, 10)

/Users/xdharmai/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


Setup for Thanos or Promethus connection

In [6]:
# URL of the route call rbac-query-proxy 
# created in namespace:open-cluster-management-observability
# after installing Observability on Advanaced Cluster Managment Install
url = "https://rbac-query-proxy-open-cluster-management-observability.apps.ocp-412-4xworker-cgk4m.dev07.red-chesterfield.com/"
# bearer token obtained from command `oc whoami -t`
token = ""
#connects to Thanos or Prometheus as dictated by the URL
pc = PrometheusConnect(url=url, headers={"Authorization": "Bearer {}".format(token)}, disable_ssl=True);

TODO: We want to be able parameterize the span of telemetry data point and window size of metrics

In [7]:
start_time=(datetime.datetime.now() - datetime.timedelta(days=2))
end_time=datetime.datetime.now()
#interval between data points gathered
step='10m'

A function to get all the clusters in  a thanos endpoint

In [8]:
def getClusters():
    clusters=set()
    q="cluster_version{}" 
    metrics = pc.custom_query(query=q)
    for metric in range(len(metrics)):
        clusters.add(metrics[metric]['metric']['cluster'])
    #print(clusters)    
    return clusters   


A function to get all the namespaces given a cluster name

In [9]:
def getNameSpaces(cluster_name):
    namespaces=set()
    q="kube_pod_container_resource_requests:sum{cluster=\""+cluster_name+"\"}" 
    metrics = pc.custom_query(query=q)
    for metric in range(len(metrics)):
        namespaces.add(metrics[metric]['metric']['namespace'])
    #print(namespaces)    
    return namespaces    


A function to get metrics given query

In [10]:
def getSnapshotMetrics(q):
    metrics = pc.custom_query(query=q)
    return metrics  

A function to get metrics given query

In [11]:
def getRangeMetrics(query,start,end,step):
    alert_total = pc.custom_query_range(
    query,
    start_time=start_time,
    end_time=end_time,
    step=step,)
    return alert_total
    

Valdiate the value and format if it exists

In [12]:
def validateDataFrame(df):
    if df.empty:
        return 0.000
    else:
        return format(float(df['value'][0]),".3f")

Main function to get CPU metrics , 
TO DO : We need similar one for memory usage

In [13]:
def getMetricsForNamespace(cluster,namespace):
    cpuRequest="sum(kube_pod_container_resource_requests:sum{cluster=\"$cluster\", resource=\"cpu\",namespace=\"$namespace\"})"

    cpuRequest=cpuRequest.replace("$cluster",cluster)
    cpuRequest=cpuRequest.replace("$namespace",namespace)
    cpuRequestMetrics= getSnapshotMetrics(cpuRequest)
    cpuRequest_df = MetricSnapshotDataFrame(cpuRequestMetrics)
    val1=validateDataFrame(cpuRequest_df)
    
    cpuUsageAvg="avg_over_time(node_namespace_pod_container:container_cpu_usage_seconds_total:sum{cluster=\"$cluster\",namespace=\"$namespace\"}[1d])"
    cpuUsageAvg=cpuUsageAvg.replace("$cluster",cluster)
    cpuUsageAvg=cpuUsageAvg.replace("$namespace",namespace)
    #print(cpuUsageAvg)
    cpuUsageAvgMetrics= getSnapshotMetrics(cpuUsageAvg)
    cpuUsageAvg_df = MetricSnapshotDataFrame(cpuUsageAvgMetrics)
    val2=validateDataFrame(cpuUsageAvg_df)

    cpuUsageMax="max_over_time(node_namespace_pod_container:container_cpu_usage_seconds_total:sum{cluster=\"$cluster\",namespace=\"$namespace\"}[1d])"
    cpuUsageMax=cpuUsageMax.replace("$cluster",cluster)
    cpuUsageMax=cpuUsageMax.replace("$namespace",namespace)
    cpuUsageMaxMetrics= getSnapshotMetrics(cpuUsageMax)
    cpuUsageMax_df = MetricSnapshotDataFrame(cpuUsageMaxMetrics)
    val3=validateDataFrame(cpuUsageMax_df)
    
    delta= format(float(val1)-float(val3),".3f")
    recommendation=format(float(val3)*1.1,".3f")
    return [cluster,namespace,val1,val2,val3,delta,recommendation]

# Main function which triggers the computation
#### Get all the clusters from the thanos DB
#### Get all the namespaces from the cluster
#### Get CPURequest and Usage for each namespace and compute recommendation



In [14]:

clusters=getClusters()
cluster_list=[['ClusterName','Namespace','CPURequest(Gi)', 'CPUUsage(Gi)', 'CPUPeak(Gi)','Delta(Gi)','Recommendation(Gi)']]
for cluster in clusters:
    ns=getNameSpaces(cluster)
    for n in ns:
        nsMetrics=getMetricsForNamespace(cluster,n)
        cluster_list.append(nsMetrics)
df=pd.DataFrame(cluster_list)
df.columns = df.iloc[0]
df = df[1:] 
df
    




,ClusterName,Namespace,CPURequest(Gi),CPUUsage(Gi),CPUPeak(Gi),Delta(Gi),Recommendation(Gi)
1,local-cluster,openshift-cluster-node-tuning-operator,0.070,0.028,0.054,0.016,0.059
2,local-cluster,open-cluster-management-hub,0.036,0.029,0.054,-0.018,0.059
3,local-cluster,openshift-insights,0.010,0.001,0.002,0.008,0.002
4,local-cluster,openshift-authentication,0.030,0.007,0.010,0.020,0.011
5,local-cluster,multicluster-engine,1.458,0.079,0.205,1.253,0.226
6,local-cluster,openshift-kube-controller-manager,1.920,0.033,0.043,1.877,0.047
7,local-cluster,openshift-multus,0.280,0.048,0.055,0.225,0.061
8,local-cluster,openshift-console,0.040,0.004,0.025,0.015,0.028
9,local-cluster,openshift-apiserver,0.330,0.032,0.103,0.227,0.113
10,local-cluster,openshift-kube-storage-version-migrator,0.010,0.000,0.000,0.010,0.000
